In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('events.csv')
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [ ]:
df.info()
# 결측치가 있는 변수는 category_code, brand, user_session이다.
# 다행히도 모든 event_type은 결측이 없어서, 사용자 행동에 대한 정보는 명확하다. (구매했는지, 구매가얼마인지 등)
# 결측치를 어떻게 처리할 것인가.
#결축이 70~80퍼센트 이상일 땐 완전 제거를 하고, 20~50% 정도일 땐 예측을 해서 채우고, 0~20퍼는 임의값으로 채운다
# event는 확실히 발생했으니, 제품 id와 카테코리 code의 NaN 값은 '기타' 로 분류하는 게 적절해 보인다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885129 entries, 0 to 885128
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     885129 non-null  object 
 1   event_type     885129 non-null  object 
 2   product_id     885129 non-null  int64  
 3   category_id    885129 non-null  int64  
 4   category_code  648910 non-null  object 
 5   brand          672765 non-null  object 
 6   price          885129 non-null  float64
 7   user_id        885129 non-null  int64  
 8   user_session   884964 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 60.8+ MB


### 1. event_time (이벤트 발생 시간)

In [ ]:
df['event_time']
# 데이터는 20년 9월 24일 부터 21년 2월 28일까지의 데이터인 것을 알 수 있다.
# 5개월 조금 넘는 기간이기 때문에, 일별, 주별, 월별 지표를 만들 수 있을 것 같다.
# dt를 활용해 어떤 요일에 구매가 자주 일어났는지 파악가능할수도?
# 시간, 분을 활용할 수 있을까? -> 웹사이트에서 하루 모든 사용자들의 체류시간 계산가능할 듯

0         2020-09-24 11:57:06 UTC
1         2020-09-24 11:57:26 UTC
2         2020-09-24 11:57:27 UTC
3         2020-09-24 11:57:33 UTC
4         2020-09-24 11:57:36 UTC
                   ...           
885124    2021-02-28 23:55:01 UTC
885125    2021-02-28 23:58:05 UTC
885126    2021-02-28 23:58:09 UTC
885127    2021-02-28 23:58:14 UTC
885128    2021-02-28 23:59:09 UTC
Name: event_time, Length: 885129, dtype: object

In [ ]:
df['event_t']=pd.to_datetime(df['event_time']).dt.date # 시간, 분까지 있는 날짜라 일자만 추출해서 날짜화
gr=df.groupby('event_t') # 날짜별 ~
gr['user_id'].nunique() # 이렇게 날짜를 활용하면 Acqusition 지표중 하나인 DAU를 만들 수 있다.

event_t
2020-09-24    1358
2020-09-25    2451
2020-09-26    2030
2020-09-27    2187
2020-09-28    2732
              ... 
2021-02-24    2904
2021-02-25    2784
2021-02-26    2960
2021-02-27    2578
2021-02-28    2775
Name: user_id, Length: 158, dtype: int64

In [ ]:
del df['event_t']

### 2. event_type (이벤트 유형)

In [ ]:
df['event_type'].value_counts(dropna=False)
# 퍼널 처럼 event가 방문 - 장바구니 - 구매로 이어질수록 점점 줄어드는 것을 확인할 수 있다.
# 가장 중요한 변수라고 생각된다. 이벤트 유형에 따라서 어떤 제품군에 관심이 있고, 실제 구매로 이어진건지 파악해야하니까.

event_type
view        793748
cart         54035
purchase     37346
Name: count, dtype: int64

### 3. product_id (제품 아이디 )

In [ ]:
print(df['product_id'].value_counts(dropna=False),'\n\n', df['product_id'].nunique(),'개의 고유 상품')
# 제품 아이디는 무수하게 많다. 갤럭시 s20, s21, s22, a 시리즈, 냉장고, 세탁기 등등..
# 나중에 일자별, 주별, 월별, 가장 잘팔린 top n 제품 아이디 추출이 가능할 것 같다.
# 인기 상품인 경우 가장 잘팔렸을 것, 상위 몇개 베스트셀러로 구분 가능할 듯 하다

product_id
1821813    14563
4099645     7089
3791351     6907
3829355     5762
1785245     5118
           ...  
303610         1
4099852        1
234297         1
429622         1
3721183        1
Name: count, Length: 53453, dtype: int64 

 53453 개의 고유 상품


### 4. category_id (카테고리 아이디)

In [ ]:
print(df['category_id'].value_counts(dropna=False),'\n\n', df['category_id'].nunique(),'개의 고유 카테고리 아이디')
# 카테코리 아이디와 카테고리 코드가 왜 구분되어 있을까?

category_id
2144415922427789416    116717
2144415925011480748     38206
2144415922528452715     36528
2144415924491387038     26604
2144415923107266682     25722
                        ...  
2144415973514412544         1
2144415972465836511         1
2144415968380584295         1
2144415937661501954         1
2144415967256510792         1
Name: count, Length: 718, dtype: int64 

 718 개의 고유 카테고리 아이디


### 5. category_code (카테코리 코드 - 범주 설명)

In [ ]:
print(df['category_code'].value_counts(dropna=False),'\n\n', df['category_code'].nunique(),'개의 고유 카테고리 아이디')
# 카테코리 코드는 아마 제품의 범주를 텍스트로 표현한 것 같다.
# 107개나 있어 가장 일별, 주별, 월별로 가장 '많이 팔린'  TOP N 제품 군, 가장 '매출액 높은'  TOP N 제품 군 파악이 가능할 것 같다.
# page_view를 가장 많이한 제품 TOP N, 장바구니 TOP N, 실 구매로 이어진 TOP N

category_code
NaN                                 236219
computers.components.videocards     116717
electronics.telephone                84360
computers.peripherals.printer        43224
stationery.cartrige                  38725
                                     ...  
furniture.bedroom.blanket               26
appliances.kitchen.refrigerators        15
furniture.kitchen.chair                 15
accessories.briefcase                   13
auto.accessories.winch                   6
Name: count, Length: 108, dtype: int64 

 107 개의 고유 카테고리 아이디


In [ ]:
df.loc[df['category_code'].fillna('other').str.contains('computer'),'category_code'].count()
# 컴퓨터 관련 상품만 316954개이다.

316954

In [ ]:
df.loc[df['category_code'].fillna('other').str.contains('furniture'),'category_code'].count()
# 전자제품 말고도 여러 잡화도 같이 파는 것 같다.

3365

In [ ]:
# 107개 제품군에 대해 전부 파악하는 건 어려울 것 같다.
# 다만 나중에 내가 원하는 지표(매출, 이벤트)에 따라 뽑을 수 있을 것 같다.
# computers.components.videocards 같은 경우 대범주 - 중범주 - 소범주로 나눠서 봐도 될 것 같긴하다
df['category_code'].str.split('.',expand=True)
# df[['main_category', 'sub_category', 'sub_sub_category']]

,0,1,2,3
0,electronics,telephone,None,None
1,computers,components,cooler,None
2,NaN,NaN,NaN,NaN
3,computers,peripherals,printer,None
4,NaN,NaN,NaN,NaN
...,...,...,...,...
885124,NaN,NaN,NaN,NaN
885125,electronics,video,tv,None
885126,electronics,clocks,None,None
885127,electronics,telephone,None,None


### 6. brand (제품 브렌드)

In [ ]:
print(df['brand'].value_counts(dropna=False),'\n\n', df['brand'].nunique(),'개의 고유 브랜드')
# 제품 카테고리와 마찬가지로 브랜드 역시 고유 브랜드가 매우 많다.

brand
NaN          212364
asus          27706
gigabyte      27673
msi           24877
palit         24802
              ...  
elekta            1
vitesse           1
vitek             1
docash            1
cipherlab         1
Name: count, Length: 1000, dtype: int64 

 999 개의 고유 브랜드


In [ ]:
df['brand'].value_counts().head(10)
# 다만 10개만 당장봐도 매우 유명한 브랜드가 나온다.
# 브랜드 역시 category_code와 비슷한 인사이트를 가져갈 수 있을 것 같다.

brand
asus         27706
gigabyte     27673
msi          24877
palit        24802
samsung      23208
amd          20110
canon        18438
panasonic    11992
pioneer      11467
sirius       11409
Name: count, dtype: int64

## brand와 category_code의 관계

In [ ]:
print('brand의 결측치는', df['brand'].isnull().sum(),'개 이고,\n'
     'category_code의 결측치는', df['category_code'].isnull().sum(),'개 입니다.' )

brand의 결측치는 212364 개 이고,
category_code의 결측치는 236219 개 입니다.


In [ ]:
# 결측치 개수가 비슷해 보인다. 서로 유의마한 관계가 있는걸까?
# 1. brand와 category_code가 동시에 결측인 경우의 event_type 파악해보자
# 2. product_id(특정 제품)을 많이 샀을까?

In [ ]:
null_brand_code=(df['category_code'].isnull()&df['brand'].isnull())
df.loc[null_brand_code].shape
# 동시에 결측인 경우의 개수는 79803개이다.

(79803, 9)

In [ ]:
# 동시 결측의 패턴을 찾아보자.
df.loc[null_brand_code,'event_type'].value_counts()
# 동시에 결측이라도,전체 데이터의 방문 - 장바구니 - 구매 비율과 엇비슷해보인다.

event_type
view        73405
cart         3569
purchase     2829
Name: count, dtype: int64

```
전체 데이터의 이벤트 타입
event_type
view        793748
cart         54035
purchase     37346
Name: count, dtype: int64
```

In [ ]:
df.loc[null_brand_code,'product_id'].value_counts().head()
# brand와 category_code가 동시에 결측인 경우, 해당하는 product_id들은?

product_id
287525     2799
246841     1964
1843522    1905
775032     1087
261142      978
Name: count, dtype: int64

In [ ]:
pr=df['product_id'].isin(['287525','246841','1843522','775032','261142'])
df.loc[pr,['category_code','brand']]
# null_brand_code, 즉 2개 변수가 전부 결측일 때 287525 제품이 속할 순 있지만,
# 287525 관련 데이터를 하나하나 보면 누락이 아닌 경우도 있을 수 있으니,
# 이들 제품 중 혹시 누락이 안된 경우가 있는지 확인했는데,
# 저 상품들(priduct_id)은 전부 제품 번호 정보만 있는 것을 확인하였다.

,category_code,brand


In [ ]:
print('전체', df['product_id'].nunique(),'개의 고유 제품 id 중,\n'
     '카테코리 코드와 브랜드가 기입되지 않은 고유 제품 id는', df.loc[null_brand_code]['product_id'].nunique() ,'개 입니다.' )

전체 53453 개의 고유 제품 id 중,
카테코리 코드와 브랜드가 기입되지 않은 고유 제품 id는 6145 개 입니다.


In [ ]:
print('전체', df['category_id'].nunique(),'개의 고유 카테코리 id 중,\n'
     '카테코리 코드와 브랜드가 기입되지 않은 카테고리 id는', df.loc[null_brand_code]['category_id'].nunique() ,'개 입니다.' )

전체 718 개의 고유 카테코리 id 중,
카테코리 코드와 브랜드가 기입되지 않은 카테고리 id는 293 개 입니다.


In [ ]:
# 결론: 큰 관련은 없어 보인다.

### 7. price (제품 가격)

In [ ]:
df['price'].describe().round(2)
# 제품 가격은 평균 146달러, 꽤 높은 평균가가 책정되어 있다 -> 고가의 전자제품

count    885129.00
mean        146.33
std         296.81
min           0.22
25%          26.46
50%          65.71
75%         190.49
max       64771.06
Name: price, dtype: float64

In [ ]:
df.loc[df['event_type']=='cart'].price.describe().round(2)
# 장바구니에 담은 제품의 가격 평균은 약 160 달러지만,

count    54035.00
mean       159.64
std        190.12
min          0.22
25%         30.16
50%         82.97
75%        226.84
max       4071.73
Name: price, dtype: float64

In [ ]:
df.loc[df['event_type']=='purchase'].price.describe().round(2)
# 장바구니에 담은 제품의 가격 평균은 약 137 달러이다.
# 구매로 이어지는 가격이 장바구니 예산선에서 내려간다고 보면 그럴싸하다.

count    37346.00
mean       137.24
std        169.93
min          0.22
25%         26.22
50%         64.48
75%        202.67
max       3717.65
Name: price, dtype: float64

### 8. user_id (사용자 아이디)

In [ ]:
print(df['user_id'].value_counts(),'\n\n',df['user_id'].nunique(), '명의 고유 고객')
# 885129의 구매 데이터 중 고유 고객이  407283명인 것을 보면,
# 전자제품을 몇번을 반복해서 사는 고객은 거의 없고(극소수의 사용자가 많은 이벤트를 발생)
# 이벤트 수가 적은 사용자가 대다수일 것이다.

user_id
1515915625554995474    572
1515915625527763086    424
1515915625591251010    363
1515915625591659523    339
1515915625537803839    329
                      ... 
1515915625541761876      1
1515915625541762343      1
1515915625541762272      1
1515915625541762044      1
1515915625611024030      1
Name: count, Length: 407283, dtype: int64 

 407283 명의 고유 고객


### 9. user_session (사용자 세션)

In [ ]:
df['user_session'].value_counts(dropna=False)

user_session
nFlhu5QzOd    572
9HqvxzKlPb    424
F6ohHpTTBU    293
iP9cILMFNS    260
BejOXRngEW    215
             ... 
q1TBz1LVkq      1
U3dPvtULrF      1
ydhD0G8EAL      1
z5YIzOJdxL      1
9pCbKMIcSx      1
Name: count, Length: 490399, dtype: int64

In [ ]:
# 상위 5개만 알아보자, 로그 데이터인 건 알겠지만, 무엇인지 감을 잡아야 할 것 같다.

In [ ]:
df.loc[df['user_session']=='nFlhu5QzOd']['event_type'].value_counts() # 상품 보기만 한 사람의 로그 기록

event_type
view    572
Name: count, dtype: int64

In [ ]:
df.loc[df['user_session']=='9HqvxzKlPb']['event_type'].value_counts() # 상품 보기만 한 사람의 로그 기록

event_type
view    424
Name: count, dtype: int64

In [ ]:
df.loc[df['user_session']=='F6ohHpTTBU']['event_type'].value_counts() # 보고, 장바구니 담고, 구매한 사람의 로그 기록

event_type
view        213
purchase     44
cart         36
Name: count, dtype: int64

In [ ]:
df.loc[df['user_session']=='iP9cILMFNS']['event_type'].value_counts() # 상품 보기만 한 사람의 로그 기록

event_type
view    260
Name: count, dtype: int64

In [ ]:
df.loc[df['user_session']=='BejOXRngEW']['event_type'].value_counts() # 보고, 장바구니 담고, 구매한 사람의 로그 기록

event_type
view        212
cart          2
purchase      1
Name: count, dtype: int64

In [ ]:
pur=df['event_type']=='purchase'
df[pur]['user_session'].value_counts()
# event_type 중에 구매가 가장 적은 것은 당연할 것이다 -> 퍼널 모양으로 전환되니까

user_session
F6ohHpTTBU    44
05CLe42L0P    36
go80Q1P74X    36
kHMr8w3r6W    28
UrgkcsrOv0    26
              ..
46WWJazHlc     1
TGx0GD1kEo     1
a9muz0RId2     1
HUnj7FrgPf     1
EqOuydyqRJ     1
Name: count, Length: 24344, dtype: int64

In [ ]:
df_ex_session=df.loc[df['user_session']=='F6ohHpTTBU']
df_ex_session.user_id.nunique()
# 숫자와 문자로 구성된 로그 데이터는, 특정 고객의 고유한 데이터값임을 확인하였다.

1

## Conclusion
- 딱히 많은 전처리할 필요는 없어 보인다. 결측치를 '기타' 로 바꾸고, 나중에 날짜별 그룹화할 때 날짜화 시키기
- 데이터가 가지는 특성들을 간단히 살펴볼 수 있었다.
- 일별, 주별, 월별, 그리고 가능하다면 시간, 분까지 날짜 데이터를 잘 groupby하는 것이 관건일 듯 하다.